In [57]:
import tensorflow as tf
from tensorflow import keras

# Univariate CNN Models

In [2]:
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        end_ix = i + n_steps
        
        if end_ix > len(sequence)-1:
            break
        
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [3]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [4]:
# reshape data as 3D array
n_features = 1
X = X.reshape(X.shape[0], X.shape[1], n_features)

In [5]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# fit the model
model.fit(X, y, epochs=2000, verbose=0)

In [7]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[101.23761]]


# Multivariate CNN Models

## Multiple Input Series

In [8]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

In [9]:
from numpy import hstack

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape(len(in_seq1), 1)
in_seq2 = in_seq2.reshape(len(in_seq2), 1)
out_seq = out_seq.reshape(len(out_seq), 1)

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset

array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [10]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps
        
        if end_ix > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [11]:
# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [12]:
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [13]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")

In [14]:
# fit the model
model.fit(X, y, epochs=2000, verbose=0)

In [15]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[205.34377]]


### Multi-headed CNN Model

In [16]:
# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [17]:
# one time series per head
n_features = 1

In [18]:
# separate input data
X1 = X[:, :, 0].reshape(X.shape[0], X.shape[1], n_features)
X2 = X[:, :, 1].reshape(X.shape[0], X.shape[1], n_features)

In [19]:
# first input model
visible1 = keras.Input(shape=(n_steps, n_features))
cnn1 = keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu")(visible1)
cnn1 = keras.layers.MaxPooling1D(pool_size=2)(cnn1)
cnn1 = keras.layers.Flatten()(cnn1)

In [20]:
# second input model
visible2 = keras.Input(shape=(n_steps, n_features))
cnn2 = keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu")(visible2)
cnn2 = keras.layers.MaxPooling1D(pool_size=2)(cnn2)
cnn2 = keras.layers.Flatten()(cnn2)

In [21]:
# merge input models
merge = keras.layers.concatenate([cnn1, cnn2])
dense = keras.layers.Dense(50, activation="relu")(merge)
output = keras.layers.Dense(1)(dense)

In [22]:
# connect input and output models
model = keras.models.Model(inputs=[visible1, visible2], outputs=output)
model.compile(optimizer="adam", loss="mse")

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 2, 64)        192         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 2, 64)        192         input_2[0][0]                    
______________________________________________________________________________________________

In [24]:
# fit the model
model.fit([X1, X2], y, epochs=1000, verbose=0)

In [25]:
# reshape one sample for making a prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape(1, n_steps, n_features)
x2 = x_input[:, 1].reshape(1, n_steps, n_features)
yhat = model.predict([x1, x2])
print(yhat)

[[206.60573]]


## Multiple Parallel Series

In [26]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps
        
        if end_ix > len(sequences)-1:
            break
        
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [27]:
# choose a number of time steps
n_steps = 3

# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)
n_features = X.shape[2]
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 3) (6, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [28]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(n_features)
])

model.compile(optimizer="adam", loss="mse")

In [29]:
# fit the model
model.fit(X, y, epochs=3000, verbose=0)

In [30]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input, verbose=0)
print(yhat)
print(yhat.shape)

[[100.528435 106.14382  206.93985 ]]
(1, 3)


In [31]:
pp = array([[70,75,145], [80,85,165]])
print(pp)
print(pp.shape)
print(pp[:, 0])

[[ 70  75 145]
 [ 80  85 165]]
(2, 3)
[70 80]


In [32]:
print(yhat[:, 0])

[100.528435]


### Multi-output CNN Model

In [33]:
# define model
visible = keras.Input(shape=(n_steps, n_features))
cnn = keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu")(visible)
cnn = keras.layers.MaxPooling1D(pool_size=2)(cnn)
cnn = keras.layers.Flatten()(cnn)
cnn = keras.layers.Dense(50, activation='relu')(cnn)

In [34]:
# define output 1
output1 = keras.layers.Dense(1)(cnn)

# define output 2
output2 = keras.layers.Dense(1)(cnn)

# define output 3
output3 = keras.layers.Dense(1)(cnn)

model = keras.models.Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer="adam", loss="mse")

In [35]:
# reshape data
y1 = y[:, 0].reshape(y.shape[0], 1)
y2 = y[:, 1].reshape(y.shape[0], 1)
y3 = y[:, 1].reshape(y.shape[0], 1)

In [36]:
# fit model
model.fit(X, [y1, y2, y3], epochs=2000, verbose=0)

In [37]:
# demonstrate prediction
x_input = array([[70,75,145], [80,85,165], [90,95,185]])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[array([[101.24718]], dtype=float32), array([[106.47648]], dtype=float32), array([[106.20636]], dtype=float32)]


# Multi-step CNN Models

In [38]:
# split a univariate sequence into samples
def split_sequence(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        end_out = end_ix + n_steps_out
        if end_out > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix], sequences[end_ix:end_out]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [39]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(5, 3) (5, 2)
[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


In [40]:
# reshape data
n_features = 1
X = X.reshape(X.shape[0], n_steps_in, n_features)

In [41]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(n_steps_out)
])

model.compile(optimizer="adam", loss="mse")

In [42]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [43]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape(1, n_steps, n_features)
yhat = model.predict(x_input)
print(yhat)

[[103.71394 116.45764]]


# Multivariate Multi-step CNN Models

### Multiple Input Multi-step Output

In [44]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        end_out = end_ix + n_steps_out -1
        if end_out > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:end_out, -1]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [45]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(6, 3, 2) (6, 2)
[[10 15]
 [20 25]
 [30 35]] [65 85]
[[20 25]
 [30 35]
 [40 45]] [ 85 105]
[[30 35]
 [40 45]
 [50 55]] [105 125]
[[40 45]
 [50 55]
 [60 65]] [125 145]
[[50 55]
 [60 65]
 [70 75]] [145 165]
[[60 65]
 [70 75]
 [80 85]] [165 185]


In [46]:
n_features = X.shape[2]

In [47]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(n_steps_out)
])

model.compile(optimizer="adam", loss="mse")

In [48]:
model.fit(X, y, epochs=2000, verbose=0)

In [49]:
# demonstrate prediction
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[185.45244 207.26752]]


### Multiple Parallel Input and Multi-step Output

In [50]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        end_out = end_ix + n_steps_out
        if end_out > len(sequences):
            break
        
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:end_out, :]
        X.append(seq_x)
        y.append(seq_y)
    
    return array(X), array(y)

In [51]:
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# convert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(5, 3, 3) (5, 2, 3)
[[10 15 25]
 [20 25 45]
 [30 35 65]] [[ 40  45  85]
 [ 50  55 105]]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [[ 50  55 105]
 [ 60  65 125]]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [[ 60  65 125]
 [ 70  75 145]]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [[ 70  75 145]
 [ 80  85 165]]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [[ 80  85 165]
 [ 90  95 185]]


In [52]:
n_features = X.shape[2]

In [53]:
# flatten output
n_out = y.shape[1] * y.shape[2]
y = y.reshape(y.shape[0], n_out)

In [54]:
# define model
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=64, kernel_size=2, activation="relu", input_shape=(n_steps, n_features)),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(n_out)
])

model.compile(optimizer="adam", loss="mse")

In [55]:
# fit model
model.fit(X, y, epochs=2000, verbose=0)

In [56]:
# demonstrate prediction
x_input = array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
x_input = x_input.reshape(1, n_steps_in, n_features)
yhat = model.predict(x_input)
print(yhat)

[[ 91.57457  97.06887 188.33896 102.80939 107.80582 210.70328]]
